<a href="https://colab.research.google.com/github/rcofrer/Coursera_Capstone/blob/master/Clustering_Toronto_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Scrapping Wikipedia Table

In [0]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
soup = BeautifulSoup(results.text, 'html.parser')

data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

## Creating DataFrame

In [33]:
df = pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


## Cleaning the DataFrame

In [0]:
df  = df[~(df['Borough']=='Not assigned')]
df  = df.loc[1:,:]

## DataFrame shape

In [72]:
df.shape

(103, 3)